In [11]:
import os

def extract_phonemes_from_lpc(directory, output_file):
    """
    Extract all unique phonemes from .lpc files and save them to a file.
    
    Args:
        directory (str): Path to the directory containing .lpc files.
        output_file (str): Path to the output file (CSV or text) where phonemes will be saved.
    """
    # Initialize a set to store unique phonemes
    unique_phonemes = set()

    # Iterate through all .lpc files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".lpc"):
            file_path = os.path.join(directory, filename)
            with open(file_path, "r") as f:
                for line in f:
                    # Extract the phoneme-syllable pair (first column)
                    phoneme_syllable = line.strip().split()[0]
                    # Split by underscore and extract the phoneme
                    syllabe = list(phoneme_syllable.split('_')[0])
                    if len(syllabe) == 3:
                        unique_phonemes.add(syllabe[0])
                        unique_phonemes.add("".join(syllabe[1:]))
                    else:
                        for phoneme in syllabe:
                            unique_phonemes.add(phoneme)

    # Save the unique phonemes to the output file
    with open(output_file, "w") as f:
        for phoneme in sorted(unique_phonemes):
            f.write(f"{phoneme}\n")

    print(f"Saved {len(unique_phonemes)} unique phonemes to {output_file}")


# Example usage
lpc_directory = "/scratch2/bsow/Documents/ACSR/data/training_videos/lpc"
output_file = "/scratch2/bsow/Documents/ACSR/data/training_videos/phoneme_dictionary.txt"
extract_phonemes_from_lpc(lpc_directory, output_file)

Saved 36 unique phonemes to /scratch2/bsow/Documents/ACSR/data/training_videos/phoneme_dictionary.txt


In [2]:
import os
import pandas as pd
import numpy as np
import re
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Load CSV files from a directory based on a filename pattern
def load_csv_files(directory, filename_pattern):
    files_data = {}
    for filename in os.listdir(directory):
        if filename_pattern in filename:
            df = pd.read_csv(os.path.join(directory, filename))
            df.dropna(inplace=True)
            base_name = filename.split(filename_pattern)[0]
            files_data[base_name] = df
    return files_data

# Load features from .npy files based on a filename pattern
def load_features(directory, filename_pattern):
    files_data = {}
    for filename in os.listdir(directory):
        if filename_pattern in filename:
            features = pd.read_csv(os.path.join(directory, filename))
            features.dropna(inplace=True)
            base_name = filename.split('_features')[0]
            files_data[base_name] = features
    return files_data

# Find corresponding phoneme files based on the base names of position filenames
def find_phoneme_files(directory, base_names):
    phoneme_files = {}
    for base_name in base_names:
        phoneme_file = os.path.join(directory, f'{base_name}.lpc')
        if os.path.exists(phoneme_file):
            phoneme_files[base_name] = phoneme_file
    return phoneme_files


In [3]:
import os
import re
import csv
import cv2
import numpy as np
import torch
import pandas as pd
import librosa
from praatio import textgrid as tgio
from tqdm import tqdm

# ==========================================================
# Helper Functions
# ==========================================================

def pad_sequences(sequences, max_length, pad_value=0):
    """
    Pad sequences to the maximum length.

    Args:
        sequences (list): List of sequences to pad.
        max_length (int): Maximum length to pad to.
        pad_value (int): Value to use for padding.

    Returns:
        np.ndarray: Padded sequences.
    """
    padded_sequences = []
    for seq in sequences:
        if len(seq) < max_length:
            padding = np.full((max_length - len(seq), seq.shape[1]), pad_value)
            padded_seq = np.vstack((seq, padding))
        else:
            padded_seq = seq[:max_length]
        padded_sequences.append(padded_seq)
    return np.array(padded_sequences)

def combine_sequences_with_padding(video_data):
    """
    Combine sequences with padding to ensure uniform length.

    Args:
        video_data (dict): Dictionary containing video data.

    Returns:
        tuple: Padded input sequences (X_student_hand_shape, X_student_hand_pos, X_student_lips, X_teacher) and padded labels (y).
    """
    max_length = max(len(video_data[video]["X_student_hand_shape"]) for video in video_data)
    
    # Pad hand shape features
    X_student_hand_shape_padded = [
        pad_sequences([video_data[video]["X_student_hand_shape"]], max_length)[0] for video in video_data
    ]
    
    # Pad hand position features
    X_student_hand_pos_padded = [
        pad_sequences([video_data[video]["X_student_hand_pos"]], max_length)[0] for video in video_data
    ]
    
    # Pad lip features
    X_student_lips_padded = [
        pad_sequences([video_data[video]["X_student_lips"]], max_length)[0] for video in video_data
    ]
    
    # Pad teacher features
    X_teacher_padded = [
        pad_sequences([video_data[video]["X_teacher"]], max_length)[0] for video in video_data
    ]
    
    # Pad labels
    y_padded = [
        video_data[video]["y"]
        + [phoneme_to_index[" "]] * (max_length - len(video_data[video]["y"]))
        for video in video_data
    ]
    
    return X_student_hand_shape_padded, X_student_hand_pos_padded, X_student_lips_padded, X_teacher_padded, y_padded

def compute_log_mel_spectrogram(audio_path, sr=16000, n_fft=400, hop_length=160, n_mels=80):
    """
    Compute the log-mel spectrogram for an audio file.

    Args:
        audio_path (str): Path to the audio file.
        sr (int): Sample rate.
        n_fft (int): FFT window size.
        hop_length (int): Hop length for STFT.
        n_mels (int): Number of mel bands.

    Returns:
        np.ndarray: Log-mel spectrogram of shape (num_frames, n_mels).
    """
    # Load audio
    audio, _ = librosa.load(audio_path, sr=sr)

    # Compute mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
    )

    # Convert to log scale
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Transpose to (num_frames, n_mels)
    log_mel_spectrogram = log_mel_spectrogram.T

    return log_mel_spectrogram

def parse_textgrid(textgrid_path):
    """
    Parse a TextGrid file to extract phoneme-level intervals.

    Args:
        textgrid_path (str): Path to the TextGrid file.

    Returns:
        list: List of (start_time, end_time, phoneme) tuples.
    """
    tg = tgio.openTextgrid(textgrid_path, includeEmptyIntervals=False)
    phone_tier = tg.getTier("phones")
    return [(start, end, label) for start, end, label in phone_tier.entries]

# Load phoneme-to-index mapping
with open(
    r"/scratch2/bsow/Documents/ACSR/data/training_videos/phoneme_dictionary.txt", "r"
) as file:
    reader = csv.reader(file)
    vocabulary_list = [row[0] for row in reader]

phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(vocabulary_list)}
index_to_phoneme = {idx: phoneme for phoneme, idx in phoneme_to_index.items()}
phoneme_to_index[" "] = len(phoneme_to_index)
index_to_phoneme[len(index_to_phoneme)] = " "

def load_coordinates(directory, base_name):
    """
    Load pre-extracted coordinates from a CSV file.

    Args:
        directory (str): Directory containing the coordinate files.
        base_name (str): Base name of the video (e.g., 'sent_01').

    Returns:
        pd.DataFrame: DataFrame containing the coordinates.
    """
    file_path = os.path.join(directory, f"{base_name}_coordinates.csv")
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)  # Drop rows with NaN values
    return df

def prepare_data_for_videos_no_sliding_windows(
    base_names, phoneme_files, audio_dir, textgrid_dir, video_dir, coordinates_dir
):
    """
    Prepare data for all videos without sliding windows.

    Args:
        base_names (list): List of base names for videos.
        phoneme_files (dict): Dictionary of phoneme file paths.
        audio_dir (str): Directory containing audio files.
        textgrid_dir (str): Directory containing TextGrid files.
        video_dir (str): Directory containing video files.
        coordinates_dir (str): Directory containing pre-extracted coordinate files.

    Returns:
        dict: Dictionary containing combined features, spectrograms, and phoneme sequences.
    """
    all_videos_data = {}
    for base_name in base_names:
        if base_name in phoneme_files:
            # Load pre-extracted coordinates
            coordinates_df = load_coordinates(coordinates_dir, base_name)
            if 'frame_number' not in coordinates_df.columns:
                raise ValueError(f"Coordinate file for {base_name} does not contain 'frame_number' column.")
            frame_numbers = coordinates_df['frame_number'].values

            # Separate coordinates into hand shape, hand position, and lip landmarks
            hand_shape_columns = [f"hand_x{i}" for i in range(21)] + [f"hand_y{i}" for i in range(21)] + [f"hand_z{i}" for i in range(21)]
            hand_pos_columns = ["hand_pos_x", "hand_pos_y", "hand_pos_z"]
            lip_columns = [f"lip_x{i}" for i in range(40)] + [f"lip_y{i}" for i in range(40)] + [f"lip_z{i}" for i in range(40)]

            X_student_hand_shape = coordinates_df[hand_shape_columns].to_numpy()
            X_student_hand_pos = coordinates_df[hand_pos_columns].to_numpy()
            X_student_lips = coordinates_df[lip_columns].to_numpy()

            # Load audio and compute spectrogram
            audio_path = os.path.join(audio_dir, f"{base_name}.wav")
            log_mel_spectrogram = compute_log_mel_spectrogram(audio_path)

            # Load TextGrid and get phoneme labels for the entire sequence
            textgrid_path = os.path.join(textgrid_dir, f"{base_name}.TextGrid")
            phoneme_intervals = parse_textgrid(textgrid_path)

            # Extract phoneme labels as a sequence (without frame-level alignment)
            phoneme_labels = [interval[2] for interval in phoneme_intervals]  # Extract phoneme labels

            # Convert phoneme labels to indices
            phoneme_indices = [phoneme_to_index.get(phoneme, -1) for phoneme in phoneme_labels]

            # Combine features, spectrogram, and phoneme indices
            all_videos_data[base_name] = {
                "X_student_hand_shape": X_student_hand_shape,  # Hand shape coordinates
                "X_student_hand_pos": X_student_hand_pos,      # Hand position coordinates
                "X_student_lips": X_student_lips,              # Lip landmarks
                "X_teacher": log_mel_spectrogram,              # Audio features (log-mel spectrogram)
                "y": phoneme_indices,                          # Phoneme labels (sequence)
            }
    return all_videos_data

In [4]:
# Directories
data_dir = r'/scratch2/bsow/Documents/ACSR/output/predictions'
phoneme_dir = r'/scratch2/bsow/Documents/ACSR/data/training_videos/lpc'
audio_dir = r'/scratch2/bsow/Documents/ACSR/data/training_videos/audio'
textgrid_dir = r'/scratch2/bsow/Documents/ACSR/data/training_videos/textgrids'
video_dir = r'/scratch2/bsow/Documents/ACSR/data/training_videos/videos'
coordinates_dir = r'/scratch2/bsow/Documents/ACSR/output/extracted_coordinates'

coordinates_data = load_csv_files(coordinates_dir, "_coordinates")
# Find phoneme files
base_names = coordinates_data.keys()
phoneme_files = find_phoneme_files(phoneme_dir, base_names)

# Prepare data
all_videos_data = prepare_data_for_videos_no_sliding_windows(
    base_names, phoneme_files, audio_dir, textgrid_dir, video_dir, coordinates_dir
)

# Combine sequences with padding
X_student_hand_shape_padded, X_student_hand_pos_padded, X_student_lips_padded, X_teacher_padded, y_padded = combine_sequences_with_padding(all_videos_data)

# Convert to PyTorch tensors
X_student_hand_shape_tensor = torch.tensor(X_student_hand_shape_padded, dtype=torch.float32)
X_student_hand_pos_tensor = torch.tensor(X_student_hand_pos_padded, dtype=torch.float32)
X_student_lips_tensor = torch.tensor(X_student_lips_padded, dtype=torch.float32)
X_teacher_tensor = torch.tensor(X_teacher_padded, dtype=torch.float32)
y_tensor = torch.tensor(y_padded, dtype=torch.long)

# Final organized data
all_videos_data = {
    "X_student_hand_shape": X_student_hand_shape_tensor,  # Hand shape coordinates
    "X_student_hand_pos": X_student_hand_pos_tensor,      # Hand position coordinates
    "X_student_lips": X_student_lips_tensor,              # Lip landmarks
    "X_teacher": X_teacher_tensor,                        # Audio features (log-mel spectrogram)
    "y": y_tensor,                                        # Phoneme labels
}

/tmp/ipykernel_3991775/2338251819.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X_student_hand_shape_tensor = torch.tensor(X_student_hand_shape_padded, dtype=torch.float32)


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Function to split data into training and validation sets
def train_val_split(data, train_ratio=0.9):
    """
    Split data into training and validation sets.

    Args:
        data (dict): Dictionary containing the dataset.
        train_ratio (float): Proportion of data to use for training.

    Returns:
        tuple: Two dictionaries for training and validation data.
    """
    num_samples = len(data['X_student_hand_shape'])
    split_idx = int(num_samples * train_ratio)
    
    # Randomize the data
    indices = torch.randperm(num_samples)
    
    # Split hand shape features
    X_student_hand_shape = data['X_student_hand_shape'][indices]
    X_student_hand_shape_train = X_student_hand_shape[:split_idx]
    X_student_hand_shape_val = X_student_hand_shape[split_idx:]
    
    # Split hand position features
    X_student_hand_pos = data['X_student_hand_pos'][indices]
    X_student_hand_pos_train = X_student_hand_pos[:split_idx]
    X_student_hand_pos_val = X_student_hand_pos[split_idx:]
    
    # Split lip features
    X_student_lips = data['X_student_lips'][indices]
    X_student_lips_train = X_student_lips[:split_idx]
    X_student_lips_val = X_student_lips[split_idx:]
    
    # Split teacher features
    X_teacher = data['X_teacher'][indices]
    X_teacher_train = X_teacher[:split_idx]
    X_teacher_val = X_teacher[split_idx:]
    
    # Split labels
    y = data['y'][indices]
    y_train = y[:split_idx]
    y_val = y[split_idx:]
    
    # Create train and validation data dictionaries
    train_data = {
        'X_student_hand_shape': X_student_hand_shape_train,
        'X_student_hand_pos': X_student_hand_pos_train,
        'X_student_lips': X_student_lips_train,
        'X_teacher': X_teacher_train,
        'y': y_train
    }
    val_data = {
        'X_student_hand_shape': X_student_hand_shape_val,
        'X_student_hand_pos': X_student_hand_pos_val,
        'X_student_lips': X_student_lips_val,
        'X_teacher': X_teacher_val,
        'y': y_val
    }
    
    return train_data, val_data


# Convert data to DataLoader format
def data_to_dataloader(data, batch_size=4, shuffle=True):
    """
    Convert data into PyTorch DataLoader format.

    Args:
        data (dict): Dictionary containing the dataset.
        batch_size (int): Batch size for the DataLoader.
        shuffle (bool): Whether to shuffle the data.

    Returns:
        DataLoader: PyTorch DataLoader object.
    """
    X_student_hand_shape_tensors = data['X_student_hand_shape']
    X_student_hand_pos_tensors = data['X_student_hand_pos']
    X_student_lips_tensors = data['X_student_lips']
    X_teacher_tensors = data['X_teacher']
    y_tensors = data['y']
    
    # Create a TensorDataset with inputs and labels
    dataset = TensorDataset(
        X_student_hand_shape_tensors,
        X_student_hand_pos_tensors,
        X_student_lips_tensors,
        X_teacher_tensors,
        y_tensors
    )
    
    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader


# Split data
train_data, val_data = train_val_split(all_videos_data)

# Prepare DataLoaders
train_loader = data_to_dataloader(train_data, batch_size=4, shuffle=True)
val_loader = data_to_dataloader(val_data, batch_size=4, shuffle=False)

print("Len of train dataset", len(train_data['X_student_hand_shape']))
print("Len of val dataset", len(val_data['X_student_hand_shape']))

# Check the DataLoader output
for batch_X_student_hand_shape, batch_X_student_hand_pos, batch_X_student_lips, batch_X_teacher, batch_y in train_loader:
    print("Batch X_student_hand_shape shape:", batch_X_student_hand_shape.shape)
    print("Batch X_student_hand_pos shape:", batch_X_student_hand_pos.shape)
    print("Batch X_student_lips shape:", batch_X_student_lips.shape)
    print("Batch X_teacher shape:", batch_X_teacher.shape)
    print("Batch y shape:", batch_y.shape)
    break

Len of train dataset 85
Len of val dataset 10
Batch X_student_hand_shape shape: torch.Size([4, 327, 63])
Batch X_student_hand_pos shape: torch.Size([4, 327, 3])
Batch X_student_lips shape: torch.Size([4, 327, 120])
Batch X_teacher shape: torch.Size([4, 327, 80])
Batch y shape: torch.Size([4, 327])


# Pretraining of a deep speech model



In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import pandas as pd

class IPADataset(Dataset):
    def __init__(self, manifest_file, alphabet_file, sample_rate=16000, n_mels=80):
        self.manifest = pd.read_csv(manifest_file, header=None)
        with open(alphabet_file, "r") as f:
            self.alphabet = f.read().splitlines()
        self.sample_rate = sample_rate
        self.n_mels = n_mels

    def __len__(self):
        return len(self.manifest)

    def __getitem__(self, idx):
        audio_path, ipa_path = self.manifest.iloc[idx]
        
        # Load audio and compute mel spectrogram
        audio, _ = librosa.load(audio_path, sr=self.sample_rate)
        mel_spec = librosa.feature.melspectrogram(
            y=audio, sr=self.sample_rate, n_mels=self.n_mels
        )
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        
        # Load IPA transcription
        with open(ipa_path, "r") as f:
            ipa = f.read().strip()
        
        # Convert IPA to indices
        ipa_indices = [self.alphabet.index(c) for c in ipa.split()]
        
        return torch.tensor(mel_spec, dtype=torch.float32), torch.tensor(ipa_indices, dtype=torch.long)

In [7]:
class DeepSpeech2(nn.Module):
    def __init__(self, num_classes, hidden_size=1024, num_layers=5, n_mels=80):
        super(DeepSpeech2, self).__init__()
        
        # 2D Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        
        # Bidirectional LSTM Layers
        self.lstm = nn.LSTM(
            input_size=32 * n_mels,  # Output size of the last convolutional layer
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for bidirectional LSTM
        
    def forward(self, x):
        
        # Convolutional Layers
        x = F.relu(self.conv1(x))
        
        x = F.relu(self.conv2(x))
        
        # Reshape for LSTM
        batch_size, channels, n_mels, time_steps = x.size()
        x = x.permute(0, 3, 1, 2)  # Move time_steps to the second dimension
        x = x.reshape(batch_size, time_steps, -1)  # Flatten the last two dimensions
        
        # LSTM Layers
        x, _ = self.lstm(x)
        
        # Fully Connected Layer
        x = self.fc(x)
        
        return x

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import librosa


def collate_fn(batch):
    """
    Collate function to pad mel spectrograms and IPA indices to the same length.
    Args:
        batch: List of tuples (mel_spec, ipa_indices).
    Returns:
        Padded mel spectrograms and IPA indices.
    """
    # Separate mel spectrograms and IPA indices
    mel_specs, ipa_indices = zip(*batch)
    
    # Transpose mel spectrograms to have time dimension first
    mel_specs = [mel_spec.T for mel_spec in mel_specs]  # Transpose to [time, n_mels]
    
    # Pad mel spectrograms to the same length
    mel_specs_padded = pad_sequence(mel_specs, batch_first=True, padding_value=0)
    
    # Pad IPA indices to the same length
    ipa_indices_padded = pad_sequence(ipa_indices, batch_first=True, padding_value=0)
    
    return mel_specs_padded, ipa_indices_padded

dataset = IPADataset(manifest_file="/scratch2/bsow/Documents/ACSR/data/train.csv", alphabet_file="/scratch2/bsow/Documents/ACSR/data/training_videos/phoneme_dictionary.txt")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [9]:
import torch.optim as optim
from torch.nn import CTCLoss

# Initialize model, optimizer, and loss function
model = DeepSpeech2(num_classes=len(phoneme_to_index))
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CTCLoss()

for epoch in range(2):  # Number of epochs
    for batch_idx, (mel_spec, ipa_indices) in enumerate(dataloader):
        # Transpose mel spectrograms back to [batch, n_mels, time]
        mel_spec = mel_spec.permute(0, 2, 1)
        
        # Forward pass
        outputs = model(mel_spec.unsqueeze(1))  # Add channel dimension
        outputs = outputs.permute(1, 0, 2)  # CTC expects (time, batch, num_classes)
        
        # Compute CTC loss
        input_lengths = torch.full((mel_spec.size(0),), outputs.size(0), dtype=torch.long)
        target_lengths = torch.tensor([len(ipa) for ipa in ipa_indices], dtype=torch.long)
        loss = criterion(F.log_softmax(outputs, dim=-1), ipa_indices, input_lengths, target_lengths)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch [{epoch+1}/10], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item()}")

Epoch [1/10], Batch [1/16], Loss: 18.781482696533203
Epoch [1/10], Batch [2/16], Loss: 3.1768174171447754
Epoch [1/10], Batch [3/16], Loss: 0.25269466638565063
Epoch [1/10], Batch [4/16], Loss: 3.3895857334136963
Epoch [1/10], Batch [5/16], Loss: 2.172699451446533
Epoch [1/10], Batch [6/16], Loss: 0.501533031463623
Epoch [1/10], Batch [7/16], Loss: 1.6183223724365234
Epoch [1/10], Batch [8/16], Loss: 1.3598785400390625
Epoch [1/10], Batch [9/16], Loss: 1.0185455083847046
Epoch [1/10], Batch [10/16], Loss: 0.7515945434570312
Epoch [1/10], Batch [11/16], Loss: 0.925859272480011
Epoch [1/10], Batch [12/16], Loss: 1.2474920749664307
Epoch [1/10], Batch [13/16], Loss: 0.8660705089569092
Epoch [1/10], Batch [14/16], Loss: 0.5709250569343567
Epoch [1/10], Batch [15/16], Loss: 0.7338602542877197
Epoch [1/10], Batch [16/16], Loss: 1.964193344116211
Epoch [2/10], Batch [1/16], Loss: 0.76752769947052
Epoch [2/10], Batch [2/16], Loss: 0.4999190866947174
Epoch [2/10], Batch [3/16], Loss: 0.80434387

In [10]:
# save the pretrained model
torch.save(model.state_dict(), "/scratch2/bsow/Documents/ACSR/output/saved_models/deepspeech2_pretrained.pth")

# Knowledge distillation

In [11]:
import torch
import torch.nn as nn

class StudentModel(nn.Module):
    def __init__(self, hand_shape_dim, hand_pos_dim, lips_dim, hidden_dim, output_dim):
        super(StudentModel, self).__init__()
        
        # Feature extractors
        self.hand_shape_extractor = nn.Sequential(
            nn.Linear(hand_shape_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )
        
        self.hand_pos_extractor = nn.Sequential(
            nn.Linear(hand_pos_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        
        self.lips_extractor = nn.Sequential(
            nn.Linear(lips_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        
        # BiLSTM layers
        self.bilstm = nn.LSTM(
            input_size=64 + 32 + 128,  # Combined feature size
            hidden_size=hidden_dim,
            num_layers=2,
            bidirectional=True,
            batch_first=True
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Output phoneme predictions

    def forward(self, hand_shape, hand_pos, lips):
        # Extract features
        hand_shape_features = self.hand_shape_extractor(hand_shape)
        hand_pos_features = self.hand_pos_extractor(hand_pos)
        lips_features = self.lips_extractor(lips)
        
        # Combine features
        combined_features = torch.cat([hand_shape_features, hand_pos_features, lips_features], dim=-1)
        
        # Pass through BiLSTM
        lstm_out, _ = self.bilstm(combined_features)
        
        # Final predictions
        output = self.fc(lstm_out)
        return output

In [14]:
def sequence_level_distillation_loss(student_logits, teacher_logits, batch_y, input_lengths, label_lengths):
    """
    Compute the sequence-level distillation loss.

    Args:
        student_logits (torch.Tensor): Logits from the student model.
        teacher_logits (torch.Tensor): Logits from the teacher model.
        batch_y (torch.Tensor): Padded ground-truth labels.
        input_lengths (torch.Tensor): Lengths of input sequences.
        label_lengths (torch.Tensor): Lengths of label sequences (excluding padding).

    Returns:
        torch.Tensor: Combined loss value.
    """
    
    # Cosine similarity loss
    cosine_loss = 1 - F.cosine_similarity(student_logits, teacher_logits, dim=-1).mean()
    
    # CTC loss
    log_probs = F.log_softmax(student_logits, dim=-1)  # Log-softmax of student_logits
    log_probs = log_probs.permute(1, 0, 2)  # Reshape to [sequence_length, batch_size, num_classes]
    
    ctc_loss = F.ctc_loss(
        log_probs,
        batch_y,  # Padded labels
        input_lengths,
        label_lengths,  # Lengths of label sequences (excluding padding)
        blank=phoneme_to_index[" "],  # Blank token index
        reduction='mean'
    )
    
    # Combine losses
    total_loss = cosine_loss + ctc_loss
    return total_loss

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader

# Initialize the student model
student_model = StudentModel(
    hand_shape_dim=63,  # 21 keypoints × 3 coordinates
    hand_pos_dim=3,     # 3 coordinates (x, y, z)
    lips_dim=120,       # 40 keypoints × 3 coordinates
    hidden_dim=256,
    output_dim=len(phoneme_to_index)  # Number of phonemes
)

# load the pretrained teacher model
teacher_model = DeepSpeech2(num_classes=len(phoneme_to_index))
teacher_model.load_state_dict(torch.load("/scratch2/bsow/Documents/ACSR/output/saved_models/deepspeech2_pretrained.pth"))

# Set teacher model to evaluation mode
teacher_model.eval()

# Optimizer
optimizer = Adam(student_model.parameters(), lr=1e-4)

def train_student_model(student_model, teacher_model, train_loader, val_loader, num_epochs=10):
    student_model.train()
    teacher_model.eval()  # Set teacher model to evaluation mode
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        
        for batch_X_student_hand_shape, batch_X_student_hand_pos, batch_X_student_lips, batch_X_teacher, batch_y in train_loader:
            # Reshape batch_X_teacher for the teacher model
            batch_X_teacher = batch_X_teacher.unsqueeze(1)  # Add channel dimension
            batch_X_teacher = batch_X_teacher.permute(0, 1, 3, 2)  # Transpose to (batch_size, 1, num_mel_bins, time_steps)
            
            # Forward pass through the student model
            student_logits = student_model(batch_X_student_hand_shape, batch_X_student_hand_pos, batch_X_student_lips)
            
            # Forward pass through the teacher model
            with torch.no_grad():
                teacher_logits = teacher_model(batch_X_teacher)  # Ensure teacher model outputs logits
            
            # Compute input_lengths
            input_lengths = torch.full(
                (batch_X_student_hand_shape.size(0),),  # Batch size
                student_logits.size(1),  # Sequence length (time steps) from student_logits
                dtype=torch.long
            )
            
            # Compute label_lengths (excluding padding)
            label_lengths = torch.tensor(
                [len(seq[seq != 42]) for seq in batch_y],  # Length of each label sequence (excluding padding)
                dtype=torch.long
            )
            
            # Compute loss
            loss = sequence_level_distillation_loss(student_logits, teacher_logits, batch_y, input_lengths, label_lengths)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader)}")
    
    print("Training complete.")

# Start training
train_student_model(student_model, teacher_model, train_loader, val_loader, num_epochs=10)

Epoch 1/10, Loss: 37.37840158289129
Epoch 2/10, Loss: 37.11240161548961
Epoch 3/10, Loss: 37.10461547157981
Epoch 4/10, Loss: 37.41879671270197
Epoch 5/10, Loss: 36.94501521370628
Epoch 6/10, Loss: 37.14860708063299
Epoch 7/10, Loss: 37.82866304570978


: 

In [56]:
print(f"batch_X_teacher shape: {batch_X_teacher.shape}")


batch_X_teacher shape: torch.Size([4, 327, 161])


In [29]:
def greedy_decoder(output, blank):
    """
    Decode model outputs using a greedy decoder.

    Args:
        output (torch.Tensor): Model outputs of shape (batch_size, sequence_length, num_classes).
        blank (int): Index of the blank token.

    Returns:
        list: List of decoded sequences.
    """
    arg_maxes = torch.argmax(output, dim=2)  # Get the most likely class for each time step
    decodes = []
    for args in arg_maxes:
        decode = []
        previous_idx = None
        for index in args:
            if index != blank and (previous_idx is None or index != previous_idx):
                decode.append(index.item())  # Append non-blank and non-repeated tokens
            previous_idx = index
        decodes.append(decode)
    return decodes


def decode_loader(model, loader, blank, index_to_phoneme):
    """
    Decode outputs for all batches in a DataLoader and return both decoded and true sequences.

    Args:
        model (torch.nn.Module): Trained model.
        loader (torch.utils.data.DataLoader): DataLoader containing input data and labels.
        blank (int): Index of the blank token.
        index_to_phoneme (dict): Mapping from indices to phonemes.

    Returns:
        tuple: (decoded_sequences, true_sequences), where:
            - decoded_sequences: List of decoded phoneme sequences.
            - true_sequences: List of true phoneme sequences.
    """
    model.eval()  # Set the model to evaluation mode
    all_decoded_sequences = []
    all_true_sequences = []

    with torch.no_grad():  # Disable gradient computation
        for X_batch, y_batch in loader:  # Iterate over batches (X_batch: inputs, y_batch: labels)
            X_batch = X_batch.to(device)  # Move inputs to device
            y_batch = y_batch.to(device)  # Move labels to device
            outputs = model(X_batch)  # Get model predictions
            decoded_phoneme_sequences = greedy_decoder(outputs, blank=blank)  # Decode outputs
            decoded_phonemes = [[index_to_phoneme[idx] for idx in sequence] for sequence in decoded_phoneme_sequences]  # Convert indices to phonemes
            all_decoded_sequences.extend(decoded_phonemes)  # Add to the list of decoded sequences

            # Convert true labels to phoneme sequences
            true_phoneme_sequences = [[index_to_phoneme[idx.item()] for idx in sequence if idx != blank and 
                                       index_to_phoneme[idx.item()] != " "] for sequence in y_batch]
            all_true_sequences.extend(true_phoneme_sequences)  # Add to the list of true sequences

    return all_decoded_sequences, all_true_sequences


# Example usage
blank_token = len(phoneme_to_index)  # Index of the blank token
decoded_train_sequences, true_train_sequences = decode_loader(model, train_loader, blank_token, index_to_phoneme)
decoded_val_sequences, true_val_sequences = decode_loader(model, val_loader, blank_token, index_to_phoneme)

# Print results
print("Decoded training phoneme sequences:", decoded_train_sequences)
print("True training phoneme sequences:", true_train_sequences)
print("Decoded validation phoneme sequences:", decoded_val_sequences)
print("True validation phoneme sequences:", true_val_sequences)



Decoded training phoneme sequences: [['la', 'p', 'ɥi', 'ʁ', 'wɑ', 'ɑ̃', 't'], ['pu', 'v', 'zɛ̃', 'ɛ', 'ʁə', 'pɛ̃', 'sa', 't'], ['mɔ̃', 'p', 'ʁɔ', 'ʁ', 't'], ['a', 't'], ['bɛ', 't', 'ʁ', 'pɛ̃', 'ɑ̃', 't'], ['dɛ', 't'], ['le', 'la', 'ka', 'do', 'pu', 'a', 't'], ['t'], ['sə', 'fɛ', 'lø', 'lø', 't'], ['la', 'p', 'lø', 'tɛ', 'də', 't'], ['i', 'p', 't'], ['pu', 'ɡo', 'n', 't', 't'], ['t'], ['i', 't'], ['pu', 'ʁa', 'ɡ', 'za', 'mɛ̃', 'ɛ', 'ma', 'nə', 't'], ['i', 't'], ['ʒə', 'a', 't'], ['t'], ['le', 'p', 'tɛ', 'kœ', 'ji', 'ʁɛ̃', 'də', 't'], ['a', 't'], ['ʁ'], ['le', 'nɔ', 'z', 'sɑ̃', 'lɛ', 'ze', 't'], ['dɛ', 't'], ['i', 't', 'ʁə', 'ze', 'na', 'tɛ', 'e'], ['la', 's', 'sɔ', 's', 't', 'ʁ', 't'], ['la', 'ya', 'sɔ', 'a', 't'], ['pu', 't'], ['le', 'ʁ'], ['lø', 't'], ['i', 'la', 'fɑ̃', 'də', 'sœ', 'ʁɔ̃', 'də', 't'], ['pu', 'zɔ̃', 'lɛ̃', 't'], ['la', 'də', 'sɔ̃', 'a', 'ti', 'a', 't'], ['la', 'də', 'sə', 't'], ['t'], ['a', 'p', 'də', 't'], ['i', 'a', 'i', 'di', 'fi', 't'], ['le', 'k', 't'], ['bɛ', 'kœ'

In [30]:
import jiwer

def calculate_per_with_jiwer(decoded_sequences, true_sequences):
    """
    Calculate the Phoneme Error Rate (PER) using jiwer.

    Args:
        decoded_sequences (list): List of decoded phoneme sequences.
        true_sequences (list): List of true phoneme sequences.

    Returns:
        float: Phoneme Error Rate (PER).
    """
    # Convert phoneme sequences to space-separated strings
    decoded_str = [" ".join(seq) for seq in decoded_sequences]
    true_str = [" ".join(seq) for seq in true_sequences]

    # Calculate PER using jiwer
    per = jiwer.wer(true_str, decoded_str)
    return per

# Example usage
train_per = calculate_per_with_jiwer(decoded_train_sequences, true_train_sequences)
val_per = calculate_per_with_jiwer(decoded_val_sequences, true_val_sequences)

print("Training PER (jiwer):", train_per, "1 - PER: ", 1 - train_per)
print("Validation PER (jiwer):", val_per, "1 - PER: ", 1 - val_per)

Training PER (jiwer): 0.84232868405094 1 - PER:  0.15767131594906003
Validation PER (jiwer): 0.9512195121951219 1 - PER:  0.04878048780487809
